In [2]:
import numpy as np
import pandas as pd
import soundfile as sf
import matplotlib.pyplot as plt
import librosa
import os

In [4]:
data_train = pd.read_csv('dataset_train.csv')
data_train = data_train.sample(frac=1).reset_index(drop=True)
data_test = pd.read_csv('dataset_test.csv')
data_test = data_test.sample(frac=1).reset_index(drop=True)

data_test_folder = "./IRMAS-TestingData/" 

def getdParam():
    dList = []
    for filename in data_train['name']:
        path = os.path.join(data_test_folder, filename + ".wav")
        audio, sr = librosa.load(path)
        dList.append(librosa.get_duration(y=audio, sr=sr))
    return dList

dmin = 5.019773242630386 # La fonction prend 1min30 à s'executer, on a donc sauvegardé le résultat dans une variable
dmean = 16.886306332321844 
dmax = 20.0
len = 2874

def getSpectre():
    L = []

    for path in data_train['path']:
        audio, sr = librosa.load(path)
        spectre = librosa.feature.melspectrogram(y = audio, sr = sr, n_mels = 128, fmax = 8000)
        ptodb = librosa.power_to_db(spectre, ref=np.max)
        L.append(ptodb)
        
    Y = np.array(data_train.drop(['name', 'path'], axis=1))    
    list_spectres = np.array(L)
    np.save('list_spectres.npy', list_spectres)
    np.save('list_spectres_values.npy', Y)

getSpectre() ##La fonction prend très longtemps à créer le fichier npy, on l'execute une fois et on enregistre le fichier
list_spectres = np.load('list_spectres.npy')
list_spectres_values = np.load('list_spectres_values.npy')
print(list_spectres.shape, list_spectres_values.shape)

(6705, 128, 130) (6705, 12)


In [5]:
X = list_spectres
Y = list_spectres_values

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=1, test_size=0.2)